# Assignment 2 
***Model solution***
* Remember that there are always more than one good way to solve these problems. Use the model solution only as a guide.
* This notebook contains each step in creating the solution, so that you can follow along. This notebook contains far more than you are expected to submit.

## Problem 1
I am using the instructions given in [examples/python package and modules.ipynb](examples/python package and modules.ipynb), specifically I will use the method that loads the package dir from inside this notebook. That way my solution notebook stays self contained and the potential for user error is minimized. 

In [1]:
import sys
sys.path.append('../mypylib')

In [2]:
from linalg import matrix as mx

Before I write my code I am reviewing the [information given in class](../linear.algebra/Gaussian_elimination.ipynb), and my first goal is to write a function that solves exactly the example given in the instructions. Then I will test this function and demonstrate how it fails. I will briefly document such failure modes in the function's doc string. 

I am also using the ipython notebook magic command that loads matplotlib and numpy into the local namespace.

In [3]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


### Setting up some test data:
First I set up a vector $\vec{u}$ as a numpy array. In order to set up the matrix I first define the column vectors `acn` where $n = 1,2,3$. Then I combine the column vectors into an array of arrays. 

In [4]:
u = array([1,4,2])

In [5]:
ar1=array([2,2,3])
ar2=array([4,5,5])
ar3=array([1,2,1])
A=array([ar1,ar2,ar3])
print(A)

[[2 2 3]
 [4 5 5]
 [1 2 1]]


The element [0,2] of A should be 3 because the standard notation is that the first index counts rows and the second counts columns.

In [6]:
A[0,2]

3

If I need the columns I can access them via the components of the transpose, e.g. the first column is:

In [7]:
A.T[0]

array([2, 4, 1])

### Matrix-vector multiplication:
In class we drafted a routine that would carry out a dot product between vectors. I add that function to my `linalg/matrix` module (this is a file called `matrix.py` in the `mypylib/linalg` dir that I have imported above). The course notes tell me that the answer of solving the set of equations should be $$\vec{v} = \pmatrix{1 \\ 1 \\ -1 }.$$ I will define that vector and then try the dot product routine on $\vec{u} \cdot \vec{v}$:

In [8]:
v = array([1,1,-1])

In [9]:
mx.dot_prod(u,v)

3

In [10]:
# I have created a doc string! Let's see if it shows ok:
mx.dot_prod?

Now we implement a matrix-vector multiplication and apply it to ${\bf A} \cdot \vec{v}$

In [11]:
# the way I do it is to proto-type it first in the notebook and then 
# copy-paste it over to matrix.py. I will leave this cell in here this 
# time, just to document how I do it

# I start with a copy of dot_prod and then modify it:
def mat_vec_prod(A,v):
    '''
    returns matrix-vector product A*v
    
    Parameters:
    -----------
    A : array of arrays
       matrix
    v : array
    
    Returns:
    --------
        array, float
        vector A*v
        
    Notes:
    ------
    https://en.wikipedia.org/wiki/Matrix_multiplication    
    '''
    
    # tests
    # len(v) == number of columns of A, which is len(A.T)    
    if not len(v) == len(A.T): 
        print('Error in matrix.mat_vec_prod: not len(vector) == len(A.T)')
        return 

    # output vector has length = numper of rows of A
    u = zeros(len(A))
    for i in range(len(u)):
        # the ith component of u is the dot product of the ith row of A
        # with the vector v
        u[i] = mx.dot_prod(A[i],v)
    return u


In [12]:
# just making sure A and v are still what I think they are
A,v

(array([[2, 2, 3],
        [4, 5, 5],
        [1, 2, 1]]), array([ 1,  1, -1]))

In [13]:
# now let's test this:
mat_vec_prod(A,v)

array([ 1.,  4.,  2.])

In [14]:
# let's try some other example, we need to make a simple case for which  
# can easiy see if it works, and that would reveal some obvious problems, 
# e.g. if I duplicate the last row of A and repeat the mat-vec product
# then I should get a length-4 vector back with the last component the 
# same as the second to last.
#
# After some experimenting I find that the following does what I want:
B = vstack((A,A[-1]))               # vstack is a numpy function!

In [15]:
# Just checking:
B

array([[2, 2, 3],
       [4, 5, 5],
       [1, 2, 1],
       [1, 2, 1]])

In [16]:
mat_vec_prod(B,v)

array([ 1.,  4.,  2.,  2.])

### Adding the new function to the module:
Now that this appears to be working I copy the `mat_vec_prod` function over to the `matrix.py` module. There are a few things to consider:
* I need to import make numpy in matrix.py
* when I have changed the matrix module and want use new things in it I need to reload the `mx` module here (you may think you can just execute the cell at the beginning that contains the `from linalg ... ` statement, but that does not work)
* when I used the `dot_prod` function inside `mat_vec_prod` I had to refer to it using the module name as prefix, i.e. `mx.dot_prod`; however, inside the `matrix` module `dot_prod` is local to `mat_vec_prod`, so I can use it directly

In [17]:
#import imp      # after changing the module that I import it needs to 
#imp.reload(mx)  # be reloaded

In [18]:
mx.mat_vec_prod(B,v)

array([ 1.,  4.,  2.,  2.])

### Gaussian elimination:

In [19]:
# reset A
ar1=array([2,2,3])
ar2=array([4,5,5])
ar3=array([1,2,1])
A=array([ar1,ar2,ar3])
AA = copy(A)  # working copy, note: AA = A is just a pointer not a copy
print(AA)

[[2 2 3]
 [4 5 5]
 [1 2 1]]


I will start by performing the first step by hand. That will give me some ideas how to compactify and generalize the algorithm into a routine.

1. divide 1$^\mathrm{st}$ equation by $a_{11} (= 2)$:

In [20]:
# I will be working with rows:
i=0
AA[i] = AA[i]/AA[i,0]

In [21]:
AA

array([[1, 1, 1],
       [4, 5, 5],
       [1, 2, 1]])

Here we can see a problem. The last element in the first row should be $1.5$. The matrix `A` has data type`dtype('int64')`. It needs to be a float. I must check for this situation and change the dtype to float. 

In [22]:
AA = A  # if the next if condition is wrong I still want to have A in the 
        # work copy AA
if not 'float' in str(AA.dtype): # there are different types of floats
    AA=A.astype(float) 
    # and integer data types, e.g. int32, int64
    # and this way I am checking in the most general way
    # I will also not overwrite A but create a new working array with the 
    # required float data type
print(AA)

[[ 2.  2.  3.]
 [ 4.  5.  5.]
 [ 1.  2.  1.]]


In [23]:
#  no try again
i=0
AA[i] = AA[i]/AA[i,0]

In [24]:
AA

array([[ 1. ,  1. ,  1.5],
       [ 4. ,  5. ,  5. ],
       [ 1. ,  2. ,  1. ]])

In [25]:
# This loop allows me to perform step 2 and 3: 
i=0
for j in range(i,len(A)): print(j)

0
1
2


In [26]:
i=0
for j in range(i+1,len(AA)):
    print (i,j)
    AA[j] -= AA[j,0]*AA[i]
    print(AA[j])

0 1
[ 0.  1. -1.]
0 2
[ 0.   1.  -0.5]


In [27]:
AA

array([[ 1. ,  1. ,  1.5],
       [ 0. ,  1. , -1. ],
       [ 0. ,  1. , -0.5]])

Now, I will repeat this with `i=1` [I discover that of course the second index in `AA[j,0]` needs to be updated as well

In [28]:
i=1
AA[i] = AA[i]/AA[i,i]
for j in range(i+1,len(AA)):
    print (i,j)
    AA[j] -= AA[j,i]*AA[i]
    print(AA[j])

1 2
[ 0.   0.   0.5]


In [29]:
# check that AA and u are still what they should be, if not 
# reset by going back to the cell:
AA,u

(array([[ 1. ,  1. ,  1.5],
        [ 0. ,  1. , -1. ],
        [ 0. ,  0. ,  0.5]]), array([1, 4, 2]))

Executing that cell again with `i=2` will give me the desired diagonal form. Now, one thing I forgot is that I need to apply the same operation to the right-hand side, i.e. $\vec{u}$. The easiest way to do this is to add $\vec{u}$ as an additional column to my working copy `AA` of the matrix `A`. I can use numpy `vstack` for that again, but add to `A.T` instead of `A`, and then take the transpose again:

In [30]:
BB = vstack((AA.T,u)).T               # vstack is a numpy function!
print(BB)

[[ 1.   1.   1.5  1. ]
 [ 0.   1.  -1.   4. ]
 [ 0.   0.   0.5  2. ]]


I believe I have everything in place to calculate the diagonal form. I will put the code fragments together as a function in the `matrix.py` module. Again, I will start with a copy of a previous function.

In [34]:
# Testing:
# import imp      
# imp.reload(mx)

In [35]:
BB,v=mx.gauss_elim(A,u)

In [36]:
BB,v

(array([[ 1. ,  1. ,  1.5,  0.5],
        [ 0. ,  1. , -1. ,  2. ],
        [ 0. ,  0. ,  1. , -1. ]]), array([ 1.,  1., -1.]))

Now we just need to implement the back-substitution

In [37]:
m  = len(A)-1     # highest row/col index (we do only square matrices)
v  = zeros(m+1,float) 
u  = BB.T[-1]       # extract RHS
AA = delete(BB,m+1,1) # recover diagonalized coefficient matrix  

In [38]:
# AA

In [39]:
v[m] = u[m]

In [40]:
v

array([ 0.,  0., -1.])

In [41]:
v[m-1] = u[m-1] - v[m]*AA[m-1,m]

In [42]:
v

array([ 0.,  1., -1.])

In [43]:
v[m-2] = u[m-2] - v[m]*AA[m-2,m] - v[m-1]*AA[m-2,m-1]

In [44]:
v

array([ 1.,  1., -1.])

This can be made into a loop. However, I realize that this can be generalized in the following way. If we go backwards and initialize `v` as `zeros()` then I may just always subtract from the u component the dot product of `v` and the respective row of `AA`:

In [45]:
for j in range(m,-1,-1): print(j)

2
1
0


In [46]:
# reset AA and u:
AA,u

(array([[ 1. ,  1. ,  1.5],
        [ 0. ,  1. , -1. ],
        [ 0. ,  0. ,  1. ]]), array([ 0.5,  2. , -1. ]))

In [47]:
for j in range(m,-1,-1):
    v[j] = u[j] - mx.dot_prod(v,AA[j])

In [48]:
v    

array([-1.5,  1. ,  0. ])

In [49]:
# add all this to function in matrix module and reload and test
imp.reload(mx)

<module 'linalg.matrix' from '../mypylib/linalg/matrix.py'>

In [50]:
u = array([1,4,2])

In [51]:
ar1=array([2,2,3])
ar2=array([4,5,5])
ar3=array([1,2,1])
A=array([ar1,ar2,ar3])
print(A)

[[2 2 3]
 [4 5 5]
 [1 2 1]]


In [52]:
mx.gauss_elim(A,u)[1]

array([ 1.,  1., -1.])